In [1]:
# set up logging
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

In [2]:
# make deterministic
from mingpt.utils import set_seed
set_seed(42)

In [3]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

In [4]:
# config
embeddingSize=504
numPoints=30 # number of points that we are going to receive to make a prediction about f given x and y
numVars=1 # the dimenstion of input points x
numYs=1 # the dimension of output points y = f(x)
blockSize = 60 # spatial extent of the model for its context
batchSize = 256
dataInfo = 'Mesh_XYSorted'

In [5]:
import math
from torch.utils.data import Dataset

class CharDataset(Dataset):

    def __init__(self, data, block_size, chars):
        data_size, vocab_size = len(data), len(chars)
        print('data has %d examples, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        
        # padding token
        self.paddingToken = '_'
        self.paddingID = self.stoi[self.paddingToken]
        self.stoi[self.paddingToken] = self.paddingID
        self.itos[self.paddingID] = self.paddingToken
        self.threshold = [-1000,1000]
        
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data # it should be a list of examples
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # grab an example from the data
        chunk = self.data[idx] # sequence of tokens including x, y, eq, etc.
        chunk = json.loads(chunk) # convert the sequence tokens to a dictionary
        
        # encode every character in the equation to an integer
        # < is SOS, > is EOS
        dix = [self.stoi[s] for s in '<'+chunk['EQ']+'>']
        inputs = dix[:-1]
        outputs = dix[1:]
        
        # add the padding to the equations
        paddingSize = max(self.block_size-len(inputs),0)
        paddingList = [self.paddingID]*paddingSize
        inputs += paddingList
        outputs += paddingList        
        
        # extract points from the input sequence
        points = torch.zeros(numVars+numYs, numPoints)
        for idx, xy in enumerate(zip(chunk['X'], chunk['Y'])):
            x = xy[0] + [0]*(max(numVars-len(xy[0]),0)) # padding
            y = [xy[1]] if type(xy[1])== float else xy[1]
            y = y + [0]*(max(numYs-len(y),0)) # padding
            p = x+y # because it is only one point 
            p = torch.tensor(p)
            #replace nan and inf
            p = torch.nan_to_num(p, nan=0.0, 
                                 posinf=self.threshold[1], 
                                 neginf=self.threshold[0])
            points[:,idx] = p
        
        """
        arrange data and targets so that the first i elements of x
        will be asked to predict the i-th element of y. Notice that
        the eventual language model will actually make block_size
        individual predictions at the same time based on this data,
        so we are being clever and amortizing the cost of the forward
        pass of the network. So for example if block_size is 4, then
        we could e.g. sample a chunk of text "hello", the integers in
        x will correspond to "hell" and in y will be "ello". This will
        then actually "multitask" 4 separate examples at the same time
        in the language model:
        - given just "h", please predict "e" as next
        - given "he" please predict "l" next
        - given "hel" predict "l" next
        - given "hell" predict "o" next
        
        In addition, because the DataLoader will create batches of examples,
        every forward/backward pass during traning will simultaneously train
        a LOT of predictions, amortizing a lot of computation. In particular,
        for a batched input of integers X (B, T) where B is batch size and
        T is block_size and Y (B, T), the network will during training be
        simultaneously training to make B*T predictions, all at once! Of course,
        at test time we can paralellize across batch B, but unlike during training
        we cannot parallelize across the time dimension T - we have to run
        a forward pass of the network to recover the next single character of the 
        sequence along each batch dimension, and repeatedly always feed in a next
        character to get the next one.
        
        So yes there is a big asymmetry between train/test time of autoregressive
        models. During training we can go B*T at a time with every forward pass,
        but during test time we can only go B at a time, T times, with T forward 
        passes.
        """
        inputs = torch.tensor(inputs, dtype=torch.long)
        outputs = torch.tensor(outputs, dtype=torch.long)
        return inputs, outputs, points


In [6]:
import json
from tqdm import tqdm
import glob
def processDataFiles(files):
    text = ''""
    for f in tqdm(files):
        with open(f, 'r') as h: 
            lines = h.read() # don't worry we won't run out of file handles
            text += lines #json.loads(line)                
    return text

In [7]:
path = 'D:\Datasets\Symbolic Dataset\Datasets\Mesh_Simple_GPT2_Sorted\TrainDatasetFixed\*.json'
files = glob.glob(path)
text = processDataFiles(files)
chars = sorted(list(set(text))+['_','T','<','>']) # extract unique characters from the text before converting the text to a list
# T is for the test data
text = text.split('\n') # convert the raw text to a set of examples
train_dataset = CharDataset(text, blockSize, chars) 

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:10<00:00,  1.00s/it]


data has 5000001 examples, 47 unique.


In [8]:
idx = np.random.randint(train_dataset.__len__())
inputs, outputs, points = train_dataset.__getitem__(idx)
print('inputs:{}'.format(inputs))
inputs = ''.join([train_dataset.itos[int(i)] for i in inputs])
outputs = ''.join([train_dataset.itos[int(i)] for i in outputs])
print('id:{}\ninputs:{}\noutputs:{}\npoints:{}'.format(idx,inputs,outputs,points))

inputs:tensor([22, 42, 35, 38,  3, 42, 40, 41, 43,  3, 44, 12,  4,  4, 33, 33, 33, 33,
        33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33,
        33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33,
        33, 33, 33, 33, 33, 33])
id:1692743
inputs:<sin(sqrt(x1))______________________________________________
outputs:sin(sqrt(x1))>______________________________________________
points:tensor([[0.0000, 0.1000, 0.2100, 0.3100, 0.4100, 0.5200, 0.6200, 0.7200, 0.8300,
         0.9300, 1.0300, 1.1400, 1.2400, 1.3400, 1.4500, 1.5500, 1.6600, 1.7600,
         1.8600, 1.9700, 2.0700, 2.9000, 3.0000, 2.1700, 2.2800, 2.3800, 2.4800,
         2.5900, 2.6900, 2.7900],
        [0.0000, 0.3100, 0.4400, 0.5300, 0.6000, 0.6600, 0.7100, 0.7500, 0.7900,
         0.8200, 0.8500, 0.8800, 0.9000, 0.9200, 0.9300, 0.9500, 0.9600, 0.9700,
         0.9800, 0.9900, 0.9900, 0.9900, 0.9900, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000]])


In [9]:
path = 'D:\Datasets\Symbolic Dataset\Datasets\Mesh_Simple_GPT2_Sorted\TestDataset\*.json'
files = glob.glob(path)
textTest = processDataFiles([files[0]])
textTest = textTest.split('\n') # convert the raw text to a set of examples
test_dataset = CharDataset(textTest, blockSize, chars)

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 334.10it/s]

data has 1001 examples, 47 unique.


In [10]:
idx = np.random.randint(test_dataset.__len__())
inputs, outputs, points = train_dataset.__getitem__(idx)
    
inputs = ''.join([train_dataset.itos[int(i)] for i in inputs])
outputs = ''.join([train_dataset.itos[int(i)] for i in outputs])
print('id:{}\ninputs:{}\noutputs:{}\npoints:{}'.format(idx,inputs,outputs,points))

id:700
inputs:<0.73*sqrt(-x1**2)__________________________________________
outputs:0.73*sqrt(-x1**2)>__________________________________________
points:tensor([[0.0000, 0.1000, 0.2100, 0.3100, 0.4100, 0.5200, 0.6200, 0.7200, 0.8300,
         0.9300, 1.0300, 1.1400, 1.2400, 1.3400, 1.4500, 1.5500, 1.6600, 1.7600,
         1.8600, 1.9700, 2.0700, 2.1700, 2.2800, 2.3800, 2.4800, 2.5900, 2.6900,
         2.7900, 2.9000, 3.0000],
        [0.0000, 0.0700, 0.1500, 0.2300, 0.3000, 0.3800, 0.4600, 0.5300, 0.6100,
         0.6800, 0.7600, 0.8400, 0.9100, 0.9800, 1.0700, 1.1400, 1.2200, 1.2900,
         1.3700, 1.4500, 1.5200, 1.5900, 1.6700, 1.7500, 1.8200, 1.9000, 1.9800,
         2.0500, 2.1300, 2.2000]])


In [11]:
from mingpt.model import GPT, GPTConfig
mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size,
                  n_layer=8, n_head=8, n_embd=512, padding_idx=train_dataset.paddingID)
model = GPT(mconf)

05/19/2021 01:16:34 - INFO - mingpt.model -   number of parameters: 2.529894e+07


In [16]:
from mingpt.trainer import Trainer, TrainerConfig

# initialize a trainer instance and kick off training
tconf = TrainerConfig(max_epochs=2, batch_size=batchSize, learning_rate=6e-6,
                      lr_decay=True, warmup_tokens=512*20, final_tokens=2*len(train_dataset)*blockSize,
                      num_workers=0, ckpt_path='./SavedModels/bestModel/checkpoint.pt')
trainer = Trainer(model, train_dataset, test_dataset, tconf)

try:
    trainer.train()
except KeyboardInterrupt:
    print('KeyboardInterrupt')

epoch 1 iter 47: train loss 0.79919. lr 5.999978e-06:   0%|                       | 48/19532 [00:20<2:17:43,  2.36it/s]

KeyboardInterrupt


In [13]:
# add a safe wrapper for numpy math functions
from numpy import *
import numpy as np

def divide(x, y):
  x = np.nan_to_num(x)
  y = np.nan_to_num(y)
  return np.divide(x,y+1e-5)

def sqrt(x):
  x = np.nan_to_num(x)
  return np.sqrt(np.abs(x)) 

# Mean square error
def mse(y, y_hat):
    y_hat = np.reshape(y_hat, [1, -1])[0]
    y_gold = np.reshape(y, [1, -1])[0]
    our_sum = 0
    for i in range(len(y_gold)):
        our_sum += (y_hat[i] - y_gold[i]) ** 2

    return our_sum / len(y_gold)

In [14]:
fName = '{}_SymbolicGPT_{}_{}_{}.txt'.format(dataInfo, 
                                             'GPTRAWEQ', 
                                             'Padding',
                                             blockSize)

In [15]:
# alright, let's sample some character-level symbolic GPT
from mingpt.utils import sample
#from gp_model import Genetic_Model
#from mlp_model import MLP_Model
    
loader = torch.utils.data.DataLoader(
                                test_dataset, 
                                shuffle=False, 
                                pin_memory=True,
                                batch_size=1,
                                num_workers=0)

testRange = [3.1,6.0]
numTestPoints = 10
#test = np.linspace(3.1,6.0,numTestPoints)

# gpm = Genetic_Model(n_jobs=-1)
# mlp = MLP_Model()

resultDict = {}
with open(fName, 'w', encoding="utf-8") as o:
    modelName = 'SymbolicGPT'
    resultDict[fName] = {modelName:[]}
    
    for i, batch in enumerate(loader):
        inputs,outputs,points = batch
        
        print('Test Case {}.'.format(i))
        o.write('Test Case {}/{}.\n'.format(i,len(textTest)))
        
        t = json.loads(textTest[i])
        
        inputs = inputs[:,0:1].to(trainer.device)
        points = points.to(trainer.device)
        outputsHat = sample(model, inputs, blockSize,
                      temperature=1.0, sample=True, 
                      top_k=10)[0]
            
        # filter out predicted
        target = ''.join([train_dataset.itos[int(i)] for i in outputs[0]])
        predicted = ''.join([train_dataset.itos[int(i)] for i in outputsHat])

        target = target.strip(train_dataset.paddingToken).split('>')
        target = target[0] if len(target[0])>1 else target[1]
        target = target.strip('<').strip(">")
        predicted = predicted.strip(train_dataset.paddingToken).split('>')
        predicted = predicted[0] if len(predicted[0])>1 else predicted[1]
        predicted = predicted.strip('<').strip(">")
       
        o.write('{}\n'.format(target))
        
        print('Target:{}\nPredicted:{}'.format(target, predicted))
        
        Ys = [] #t['YT']
        Yhats = []
        for xs in t['XT']:
            try:
                eqTmp = target + '' # copy eq
                eqTmp = eqTmp.replace(' ','')
                eqTmp = eqTmp.replace('\n','')
                for i,x in enumerate(xs):
                    # replace xi with the value in the eq
                    eqTmp = eqTmp.replace('x{}'.format(i+1), str(x))
                    if ',' in eqTmp:
                        assert 'There is a , in the equation!'
                YEval = eval(eqTmp)
                YEval = 0 if np.isnan(YEval) else YEval
                YEval = 100 if np.isinf(YEval) else YEval
            except:
                YEval = 100 #TODO: Maybe I have to punish the model for each wrong template
            Ys.append(YEval)
            try:
                eqTmp = predicted + '' # copy eq
                eqTmp = eqTmp.replace(' ','')
                eqTmp = eqTmp.replace('\n','')
                for i,x in enumerate(xs):
                    # replace xi with the value in the eq
                    eqTmp = eqTmp.replace('x{}'.format(i+1), str(x))
                    if ',' in eqTmp:
                        assert 'There is a , in the equation!'
                Yhat = eval(eqTmp)
                Yhat = 0 if np.isnan(Yhat) else Yhat
                Yhat = 100 if np.isinf(Yhat) else Yhat
            except:
                Yhat = 100
            Yhats.append(Yhat)
        mseErr = mse(Ys,Yhats)
        
        if type(mseErr) is np.complex128:
            mseErr = mseErr.real
            
        resultDict[fName][modelName].append(mseErr)
        
        o.write('{}:{}\n{}\n\n'.format(modelName, 
                               mseErr,
                               predicted))
        
        print('MSE:{}\n'.format(mseErr))

Test Case 0.
Target:-sin(1.1*x1+0.64)
Predicted:sqrt(-sin(0.3*x1))
MSE:0.778124934902231

Test Case 1.
Target:-1.42*x1+sqrt(x1+0.53)
Predicted:0.85*sqrt(x1**2+0.52*x1)
MSE:71.99488528437138

Test Case 2.
Target:sqrt(-sin(0.2*x1))
Predicted:0.28*x1**2+sin(0.73*x1)
MSE:27.408434818121684

Test Case 3.
Target:sin(sqrt(x1))
Predicted:sqrt(x1**2)
MSE:14.754285680325067

Test Case 4.
Target:0.86*x1**2-0.59*x1+1.36
Predicted:sin(sin(x1+0.74))
MSE:345.553567173615

Test Case 5.
Target:0.28*sqrt(-x1**2)
Predicted:0.61*sqrt(x1)
MSE:0.014295335700979857

Test Case 6.
Target:sin(0.07*x1**2+0.28*x1)
Predicted:0.64*x1**2+0.18*x1
MSE:238.09210116644783

Test Case 7.
Target:sin(x1)
Predicted:0.87*x1**(1/4)
MSE:3.8078208783262633

Test Case 8.
Target:0.81*sqrt(-x1**2-0.95*x1+0.12)
Predicted:-sin(0.29*x1**2-0.22*x1)
MSE:17.87784156061148

Test Case 9.
Target:sin(x1+0.37)
Predicted:sin(sqrt(x1))
MSE:2.348341143969906

Test Case 10.
Target:-0.68*x1**2+1.77*x1
Predicted:sqrt(x1)
MSE:91.36939809002749

Test

KeyboardInterrupt: 